# NFL Fantasy Football 

# 0. Import libraries

In [1]:
# Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import re
import requests
from bs4 import BeautifulSoup

# Own libraries
from library_nfl import *

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Show all columns in pandas
pd.set_option('display.max_columns', 500) 

# Graphing style
plt.style.use('seaborn-colorblind')

%matplotlib inline

# 1. Scrape data

## 1.1 Player stats

In [2]:
# Links for data in each position
urls_22 = ['https://www.fantasypros.com/nfl/stats/qb.php?year=2022&scoring=PPR',
           'https://www.fantasypros.com/nfl/stats/rb.php?year=2022&scoring=PPR',
           'https://www.fantasypros.com/nfl/stats/wr.php?year=2022&scoring=PPR',
           'https://www.fantasypros.com/nfl/stats/te.php?year=2022&scoring=PPR',
           'https://www.fantasypros.com/nfl/stats/k.php?year=2022&scoring=PPR',
           'https://www.fantasypros.com/nfl/stats/dst.php?year=2022&scoring=PPR',
           'https://www.fantasypros.com/nfl/stats/db.php?year=2022&scoring=PPR']

urls_21 = ['https://www.fantasypros.com/nfl/stats/qb.php?year=2021&scoring=PPR',
           'https://www.fantasypros.com/nfl/stats/rb.php?year=2021&scoring=PPR',
           'https://www.fantasypros.com/nfl/stats/wr.php?year=2021&scoring=PPR',
           'https://www.fantasypros.com/nfl/stats/te.php?year=2021&scoring=PPR',
           'https://www.fantasypros.com/nfl/stats/k.php?year=2021&scoring=PPR',
           'https://www.fantasypros.com/nfl/stats/dst.php?year=2021&scoring=PPR',
           'https://www.fantasypros.com/nfl/stats/db.php?year=2021&scoring=PPR']

urls_20 = ['https://www.fantasypros.com/nfl/stats/qb.php?year=2020&scoring=PPR',
           'https://www.fantasypros.com/nfl/stats/rb.php?year=2020&scoring=PPR',
           'https://www.fantasypros.com/nfl/stats/wr.php?year=2020&scoring=PPR',
           'https://www.fantasypros.com/nfl/stats/te.php?year=2020&scoring=PPR',
           'https://www.fantasypros.com/nfl/stats/k.php?year=2020&scoring=PPR',
           'https://www.fantasypros.com/nfl/stats/dst.php?year=2020&scoring=PPR',
           'https://www.fantasypros.com/nfl/stats/db.php?year=2020&scoring=PPR']

# Import 2022 data
data_stats_22 = pull_data_stats(urls_22)

# Import 2021 data
data_stats_21 = pull_data_stats(urls_21)

# Import 2021 data
data_stats_20 = pull_data_stats(urls_20)


In [3]:
data_stats_22['RB'].head(4)

,rank,player,att,yds,y/a,lg,20+,td,rec,tgt,yds,y/r,td,fl,g,fpts,fpts/g,pos,team
0,1,Austin Ekeler,204,915,4.5,72,10,13,107,127,722,6.7,5,3,17,372.7,21.9,RB,LAC
1,2,Christian McCaffrey,244,1139,4.7,49,14,8,85,108,741,8.7,5,0,17,356.4,21.0,RB,SF
2,3,Josh Jacobs,340,1653,4.9,86,14,12,53,64,400,7.5,0,1,17,328.3,19.3,RB,LV
3,4,Derrick Henry,349,1538,4.4,56,20,13,33,41,398,12.1,0,3,16,302.8,18.9,RB,TEN


## 1.2 Snap count

In [4]:
# URLS for last 3 years

urls_22 = ['https://www.fantasypros.com/nfl/reports/snap-counts/qb.php?year=2022&show=perc',
           'https://www.fantasypros.com/nfl/reports/snap-counts/rb.php?year=2022&show=perc',
           'https://www.fantasypros.com/nfl/reports/snap-counts/wr.php?year=2022&show=perc',
           'https://www.fantasypros.com/nfl/reports/snap-counts/te.php?year=2022&show=perc']

urls_21 = ['https://www.fantasypros.com/nfl/reports/snap-counts/qb.php?year=2021&show=perc',
           'https://www.fantasypros.com/nfl/reports/snap-counts/rb.php?year=2021&show=perc',
           'https://www.fantasypros.com/nfl/reports/snap-counts/wr.php?year=2021&show=perc',
           'https://www.fantasypros.com/nfl/reports/snap-counts/te.php?year=2021&show=perc']

urls_20 = ['https://www.fantasypros.com/nfl/reports/snap-counts/qb.php?year=2020&show=perc',
           'https://www.fantasypros.com/nfl/reports/snap-counts/rb.php?year=2020&show=perc',
           'https://www.fantasypros.com/nfl/reports/snap-counts/wr.php?year=2020&show=perc',
           'https://www.fantasypros.com/nfl/reports/snap-counts/te.php?year=2020&show=perc']


data_snapcount_22 = pull_data_snapcount(urls_22)
data_snapcount_21 = pull_data_snapcount(urls_21)
data_snapcount_20 = pull_data_snapcount(urls_20)

## Snapcount clean

In [5]:
data_snapcount_22['RB']

,player,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,ttl,avg_snap_pct_played,avg_snap_pct_global,pos
0,Mark Ingram II,NO,33.0,37.0,33.0,48.0,22.0,33.0,28.0,8.0,0.0,0.0,0.0,30.0,42.0,NaN,0.0,0.0,0.0,0.0,205.0,31.400000,18.470588,RB
1,Brandon Bolden,LV,28.0,0.0,36.0,7.0,16.0,NaN,0.0,0.0,7.0,0.0,5.0,0.0,0.0,2.0,0.0,4.0,21.0,0.0,81.0,14.000000,7.411765,RB
2,Cordarrelle Patterson,ATL,65.0,59.0,61.0,29.0,0.0,0.0,0.0,0.0,39.0,38.0,49.0,58.0,46.0,NaN,50.0,37.0,50.0,28.0,382.0,46.846154,35.823529,RB
3,Giovani Bernard,TB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,45.0,25.0,45.000000,2.647059,RB
4,Rex Burkhead,HOU,71.0,37.0,41.0,32.0,21.0,NaN,35.0,19.0,22.0,25.0,0.0,18.0,0.0,8.0,19.0,16.0,19.0,15.0,251.0,26.533333,23.411765,RB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,Malik Davis,DAL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33.0,NaN,13.0,18.0,0.0,4.0,0.0,0.0,0.0,32.0,9.0,78.0,18.166667,6.411765,RB
156,Jaylen Warren,PIT,37.0,29.0,20.0,25.0,51.0,31.0,23.0,29.0,NaN,43.0,4.0,0.0,20.0,40.0,34.0,31.0,40.0,42.0,342.0,31.187500,29.352941,RB
157,Isiah Pacheco,KC,23.0,9.0,8.0,21.0,3.0,15.0,30.0,NaN,22.0,56.0,40.0,51.0,45.0,43.0,35.0,49.0,45.0,36.0,351.0,31.235294,31.235294,RB
158,Julius Chestnut,TEN,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,13.0,10.0,2.0,0.0,22.0,0.0,29.0,11.750000,2.764706,RB


## 1.3 Team's target distribution by position

In [6]:
# List of abbreviations
team_abbreviations = ['KC', 'BUF', 'PHI', 'CIN', 'SEA', 'CHI', 'MIN', 'JAC', 'NYG',
                    'DET', 'LAC', 'FA', 'NYJ', 'BAL', 'MIA', 'DEN', 'NO', 'DAL', 'ARI',
                    'HOU', 'CAR', 'NE', 'WAS', 'LV', 'TEN', 'PIT', 'TB', 'ATL', 'LAR',
                    'SF', 'CLE', 'IND', 'GB']
# List of full names 
team_names = ['Kansas City Chiefs', 'Buffalo Bills', 'Philadelphia Eagles',
            'Cincinnati Bengals', 'Seattle Seahawks', 'Chicago Bears',
            'Minnesota Vikings', 'Jacksonville Jaguars', 'New York Giants',
            'Detroit Lions', 'Los Angeles Chargers', 'Free Agent', 'New York Jets',
            'Baltimore Ravens', 'Miami Dolphins', 'Denver Broncos', 'New Orleans Saints',
            'Dallas Cowboys', 'Arizona Cardinals', 'Houston Texans',
            'Carolina Panthers', 'New England Patriots', 'Washington Commanders',
            'Las Vegas Raiders', 'Tennessee Titans', 'Pittsburgh Steelers',
            'Tampa Bay Buccaneers', 'Atlanta Falcons', 'Los Angeles Rams',
            'San Francisco 49ers', 'Cleveland Browns', 'Indianapolis Colts',
            'Green Bay Packers']

# Create the corrected dictionary
nfl_teams_dict = dict(zip(team_names, team_abbreviations))

In [7]:
urls_22 = ['https://www.fantasypros.com/nfl/reports/targets-distribution/?year=2022&start=1&end=18']

urls_21 = ['https://www.fantasypros.com/nfl/reports/targets-distribution/?year=2021&start=1&end=18']

urls_20 = ['https://www.fantasypros.com/nfl/reports/targets-distribution/?year=2020&start=1&end=18']

df_target_dist_22 = pull_data_target_distribution(urls_21, nfl_teams_dict)
df_target_dist_21 = pull_data_target_distribution(urls_21, nfl_teams_dict)
df_target_dist_20 = pull_data_target_distribution(urls_21, nfl_teams_dict)

In [8]:
df_target_dist_22.head(4)

,team_name,wr_targets,wr_pct,rb_targets,rb_pct,te_targets,te_pct,total_targets,team
0,Arizona Cardinals,359,63.0,103,18.1,108,18.9,570,ARI
1,Atlanta Falcons,251,45.2,146,26.3,158,28.5,555,ATL
2,Baltimore Ravens,337,56.9,83,14.0,172,29.1,592,BAL
3,Buffalo Bills,443,71.2,96,15.4,83,13.3,622,BUF


# 2. RB Data cleaning

In [14]:
def create_data_rb(stats_data, snap_data, targets_data):
    # Get initial stats dataframe
    df_rb = stats_data['RB']

    # Get snapcount data
    df_snaps = snap_data['RB'][['player', 'avg_snap_pct_played', 'avg_snap_pct_global']]

    # Get targets dataframe by team
    df_targets = targets_data[['team', 'rb_targets', 'rb_pct', 'total_targets']]

    # Merge snaps dataframe on Player 
    df_rb = df_rb.merge(df_snaps, how='left', on='player')

    # Merge snaps dataframe on Player 
    df_rb = df_rb.merge(df_targets, how='left', on='team')

    # Return data
    return df_rb


## Create data for RB

In [16]:
df_rb_22 = create_data_rb(data_stats_22, data_snapcount_22, df_target_dist_22)
df_rb_21 = create_data_rb(data_stats_21, data_snapcount_21, df_target_dist_21)
df_rb_20 = create_data_rb(data_stats_20, data_snapcount_20, df_target_dist_20)